In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Data preparation

In [ ]:
import pandas as pd
import numpy as np

def load_data(csv_file):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Extract the pixel data from the DataFrame
    pixel_data = df.values
#     print(pixel_data[0].shape[0]-1)
#     Reshape the pixel data to images
    images = []
    labels = []
    for pixels in pixel_data:
        # Reshape the pixel data to the original image shape (e.g., 224x224x3 for RGB images)
        image = pixels[1:].reshape((28, 28, 1))
        label = pixels[0]
        # Append the image to the list of images
        images.append(image)
        labels.append(label)
    
    return images, labels

## 1.1 Load data


In [ ]:
data_path_train = '/kaggle/input/digit-recognizer/train.csv'
data_path_test = '/kaggle/input/digit-recognizer/test.csv'

# Load the data
train = pd.read_csv(data_path_train)
test = pd.read_csv(data_path_test)

In [ ]:
train.head()

In [ ]:
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

Y_train.value_counts()

## 1.2 Normalization
perform a grayscale normalization to reduce the effect of illumination's differences.

In [ ]:
# Normalize the data
Y_train = Y_train / 255.0
X_train = X_train / 255.0

## 1.3 Reshape

In [ ]:
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

## 1.4 Label encoding

In [ ]:
# Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from tensorflow.keras.utils import to_categorical  # convert to one-hot-encoding

Y_train = to_categorical(Y_train, num_classes = 10)

## 1.5 Split training and valdiation set

In [ ]:
# Split the train and the test set
from sklearn.model_selection import train_test_split
X_train_, X_test, Y_train_, Y_test = train_test_split(X_train, Y_train, test_size = 0.20)

In [ ]:
print(X_train_.shape)
print(X_test.shape)


In [ ]:
# Split the train and the validation set for the fitting
X_test, X_val, Y_test, Y_val = train_test_split(X_test, Y_test, test_size = 0.40)

In [ ]:
print(X_test.shape)
print(X_val.shape)

## 1.6 Data augmentation

In [ ]:
# Hide cell in viewer


# Generate images
from tensorflow.keras.preprocessing.image import ImageDataGenerator


def augment_images(images, rotation_range=20, width_shift_range=0.1,
                   height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                   horizontal_flip=True, fill_mode='nearest'):
    # Create an ImageDataGenerator object with the specified augmentations
    datagen = ImageDataGenerator(
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        shear_range=shear_range,
        zoom_range=zoom_range,
        horizontal_flip=horizontal_flip,
        fill_mode=fill_mode
    )

    # Fit the ImageDataGenerator object to your data (optional)
    datagen.fit(images)

    # Generate augmented images
    augmented_images = []
    for img in images:
        augmented_img = next(datagen.flow(np.expand_dims(img, axis=0), batch_size=1))[0]
        augmented_images.append(augmented_img.squeeze())

    return augmented_images



In [ ]:
# Hide cell in viewer
train_generator = augment_images(X_train)


In [ ]:
# Hide cell in viewer
len(train_generator)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train_)

## 3. CNN : Define the model

In [ ]:
## Model creation
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from keras.layers import Dense, BatchNormalization

model = Sequential([
     Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)),
     Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.25),
    Flatten(),
    Dense(256, activation = "relu"),
    Dropout(0.5),
    Dense(10, activation = "softmax")
    
      ])
# Set optimizer
# Compile the model
model.compile(optimizer='adam', loss = 'categorical_crossentropy' ,
              metrics= ['accuracy'])
# Summary about the Model
model.summary()

## 4- Training the modle

In [ ]:
# Train the model # Fit the model

history = model.fit(datagen.flow(X_train_,Y_train_, batch_size=64),
                              epochs = 5, validation_data = (X_val,Y_val))

you can use just 2 epochs

## 5- Evaluate the model

In [ ]:

test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print("Test Accuracy:", test_accuracy)

In [ ]:
# predict results
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("cnn_mnist_datagen.csv",index=False)

In [ ]:
y_pred = model.predict(test)
y_pred = np.argmax(y_pred,axis=1)


In [ ]:
submit = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
submit.Label = results
submit.to_csv('submission.csv',index=False)